# Example 1: Getting Started

In which we learn about indicators and models

In [2]:
import robin_stocks as r
import pandas as pd

# logging into robinhood
r.login()

In [3]:
# setting watchlist
basket = ['FB','AAPL','TSLA','NFLX']

In [4]:
# getting data
data = r.get_historicals(basket[0], span='year')
df = pd.DataFrame(data)
df.head()

,begins_at,open_price,close_price,high_price,low_price,volume,session,interpolated,symbol
0,2019-07-18T00:00:00Z,200.150000,200.780000,202.470000,199.380000,11956554,reg,False,FB
1,2019-07-19T00:00:00Z,202.180000,198.360000,202.330000,198.070000,12098262,reg,False,FB
2,2019-07-22T00:00:00Z,199.910000,202.320000,202.570000,198.810000,13589024,reg,False,FB
3,2019-07-23T00:00:00Z,202.840000,202.360000,204.242000,200.962500,14583680,reg,False,FB
4,2019-07-24T00:00:00Z,197.630000,204.660000,204.810000,197.220000,32532540,reg,False,FB


indicators.py contains functions that all take a dataframe and some kwargs as inputs, and then add a column to the dataframe 

In [5]:
# adding indicators
from indicators import add_moving_average, add_RSI
add_moving_average(df, n=200)
add_moving_average(df, n=10)
add_RSI(df, n=2)

In [6]:
# creating a model
from models import Mean_Reversion
'''
from models.py:
implements a mean reversion strategy
says buy if 1) price is higher than the 200_period_ma and 2) the 2_period_RSI is < 10
says sell if price is lower than 10_period_ma
'''
strat = Mean_Reversion('MR', n_for_long_ma=200, n_for_short_ma=10, n_for_RSI=2)

In [8]:
# validating input
strat.check_input(df)

True

In [10]:
# does the model think we should buy Facebook stock right now?
strat.buy_signal(df)

False

In [11]:
# does the model think we should sell Facebook stock right now (assuming we have any)?
strat.sell_signal(df)

False

# Example 2: Trading

Introducing the world and investor classes

In [12]:
# build world
from indicators import Moving_Average, RSI
from world import world_from_live
'''
from world.py:
The world class is the 'environment' within which the investor/model classes operate
this layer of abstraction allows changing from backtesting to livetrading and back without
changing the code that makes decisions
'''
w = world_from_live(basket, cash=10000, indicators=[Moving_Average(n=200), Moving_Average(n=10), RSI(2)])

In [13]:
# get model
from models import Mean_Reversion
mr = Mean_Reversion('Mean Reversion', 200, 10, 2)

In [14]:
# build investor
from investor import Investor
'''
the investor class takes a list of models, a world, some settings, and generates ideas
for concrete trades to execute
'''
investor = Investor(models=[mr], world=w, live=False)

In [15]:
# livetrade with investor
investor.routine()

running routine, found 0 trades


# Example 3: Backtesting

Testing models on historical data

In [16]:
# build base world
from indicators import Moving_Average, RSI
from world import world_from_live
w = world_from_live(basket, cash=10000, indicators=[Moving_Average(n=200), Moving_Average(n=10), RSI(2)])

In [17]:
# get model
from models import Mean_Reversion
mr = Mean_Reversion('Mean Reversion', 200, 10, 2)

In [18]:
# build investor
from investor import Investor
i = Investor(models=[mr], world=w, live=False)

In [19]:
# set up backtest
from backtest import Backtest
b = Backtest(name='mean_reversion_backtest_000', investor=i, base_world=w)

In [ ]:
# do backtest and print results

# COMING SOON